In [1]:
# nostri import 
import random
import pickle

import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import tensorflow_text #necessaria per hub.load

In [2]:
df = pd.read_csv('/data/NLP_Attack/server_code/amazon/Reviews.csv')

data_sinonimi = pd.read_csv('/data/NLP_Attack/server_code/amazon/sinonimi/amazon_sinonimi.csv')
data_embedding = pd.read_csv('/data/NLP_Attack/server_code/amazon/embedding/amazon_embedding.csv')

In [3]:
df["review_type"] = df["Score"].apply(lambda x: "negative" if x < 4 else "positive")

In [4]:
print(df.shape)
print(data_sinonimi.shape)
print(data_embedding.shape)

(568454, 11)
(152598, 4)
(149967, 4)


In [14]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [17]:
type_one_hot = OneHotEncoder(sparse=False).fit_transform(
  df.review_type.to_numpy().reshape(-1, 1)
)

In [ ]:
train_reviews, test_reviews, y_train, y_test =\
  train_test_split(
    df.Text,
    type_one_hot,
    test_size=.3,
    random_state=RANDOM_SEED
  )

In [5]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [6]:
model = keras.Sequential()

model.add(keras.layers.Input(shape=(1,), dtype=tf.string))
model.add(keras.layers.Lambda(lambda x: tf.squeeze(tf.cast(x, tf.string))))
model.add(hub.KerasLayer(handle=embed,output_shape=512)) # pre trained Convolutional Neural Net. 
model.add(keras.layers.Dense(units=256, activation='relu')) 
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              None                      0         
_________________________________________________________________
keras_layer (KerasLayer)     (None, 512)               85213184  
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2

In [7]:
model.load_weights('model.h5')

In [8]:
from sklearn.preprocessing import OneHotEncoder

df_one_hot = OneHotEncoder(sparse=False).fit_transform(
  df.review_type.to_numpy().reshape(-1, 1)
)

sinonimi_one_hot = OneHotEncoder(sparse=False).fit_transform(
  data_sinonimi.review_type.to_numpy().reshape(-1, 1)
)

embedding_one_hot = OneHotEncoder(sparse=False).fit_transform(
  data_embedding.review_type.to_numpy().reshape(-1, 1)
)

In [ ]:
result = model.evaluate(df.Text, df_one_hot)

In [11]:
print(f'Accuracy: {result[1]}')

Accuracy: 0.8586974740028381


In [ ]:
result = model.evaluate(data_sinonimi.Text, sinonimi_one_hot)

In [13]:
print(f'Accuracy: {result[1]}')

Accuracy: 0.7029646635055542


In [ ]:
result = model.evaluate(data_embedding.Text, embedding_one_hot)

In [24]:
print(f'Accuracy: {result[1]}')

Accuracy: 0.8409450054168701
